# Turbulent Transfer

Setup the gold files for turblent transfer through a random number generation. Run both the IPW `trbxfr` which implments the `hle1` calculation and the python `libsnobal.hle1`.

In [ ]:
import pandas as pd
import numpy as np
import subprocess as sp
import os

from pysnobal import libsnobal
from pysnobal.constants import FREEZE

def calculate_hle1(inputs):

    P_a = libsnobal.hysat(
        libsnobal.SEA_LEVEL,
        libsnobal.STD_AIRTMP,
        libsnobal.STD_LAPSE,
        inputs['elevation'] / 1000.0,
        libsnobal.GRAVITY,
        libsnobal.MOL_AIR)

    H, L_v_E, E, status = libsnobal.hle1(
        P_a,
        inputs['ta'],
        inputs['ts'],
        inputs['za'],
        inputs['ea'],
        inputs['es'],
        inputs['za'],
        inputs['u'],
        inputs['zu'],
        inputs['z0']
    )

    return H, L_v_E, E, status

In [ ]:
# Generate random inputs
nvalues = 100

df = pd.DataFrame(columns=['elevation', 'ta', 'ts', 'za',
                           'ea', 'es', 'u', 'zu', 'z0'])

df['elevation'] = np.random.randint(100, 4000, nvalues)
df['ta'] = np.random.randint(-40, 40, nvalues) + FREEZE
df['ts'] = np.random.randint(-40, 0, nvalues) + FREEZE
df['za'] = np.random.randint(2, 5, nvalues)

df['ea'] = np.random.randint(40, 7000, nvalues)
sat_vp = libsnobal.sati_np(df['ta'])
df.loc[df.ea > sat_vp, 'ea'] = sat_vp[df.ea > sat_vp]
df['es'] = libsnobal.sati_np(df['ts'])

df['u'] = np.random.randint(1, 40, nvalues)
df['zu'] = np.random.randint(3, 5, nvalues)
df['z0'] = np.random.randint(5, 100, nvalues)/1000


In [ ]:
# Run the pysnobal hle1

for index, row in df.iterrows():
    H, L_v_E, E, status = calculate_hle1(row.to_dict())

    df.loc[index, 'py_H'] = H
    df.loc[index, 'py_L_v_E'] = L_v_E
    df.loc[index, 'py_E'] = E
    df.loc[index, 'py_status'] = status

df
    

In [ ]:
# Run trbxfr from IPW to ensure it's close

cmd_str = 'echo "{} {} {} {} {} {} {} {}" | trbxfr -z {} -d -m'

for index, row in df.iterrows():

    cmd = cmd_str.format(
        row['zu'],
        row['za'],
        row['z0'],
        row['ta'],
        row['ts'],
        row['ea'],
        row['es'],
        row['u'],
        row['elevation']
    )
    print(cmd)
    print(os.environ['PATH'])
    out = sp.check_output(cmd, shell=True, universal_newlines=True)

    print(out)
    break

    R = np.ndarray((6, 1))
    for i, m in enumerate(c):
        R[i] = float(m.rstrip().split(' ')[-1])